In [1]:
import requests
from datetime import datetime
import pandas as pd
from tqdm.notebook import tqdm

pd.options.mode.chained_assignment = None  # default='warn'

In [175]:
urls_raw = pd.read_excel('404_URLs_240611_MasterList.xlsx')

In [167]:
urls_raw

,URL,Visits
0,https://www.samsung.com/us/mobile/phones/galax...,NaN
1,https://www.samsung.com/us/mobile/phones/galax...,NaN
2,https://www.samsung.com/us/mobile/mobile-acces...,NaN
3,https://www.samsung.com/us/mobile/mobile-acces...,NaN
4,https://www.samsung.com/us/smartphones/galaxy-...,NaN
...,...,...
494,https://www.samsung.com/us/smartphones/galaxy-...,NaN
495,https://www.samsung.com/us/smartphones/galaxy-...,NaN
496,https://www.samsung.com/us/smartphones/galaxy-...,NaN
497,https://www.samsung.com/us/app/smartphones/gal...,NaN


In [178]:
urls = pd.DataFrame()
urls.loc[:,'URL'] = urls_raw.loc[:,'URL']
urls.loc[:,'Visits'] = urls_raw.loc[:,'Visits']

# Visit count limit
urls = urls[urls['Visits']>=5]

In [179]:
urls.insert(2,'response_code_1','none')
urls.insert(3,'redirect_url_1','none')
urls.insert(4,'response_code_2','none')
urls.insert(5,'redirect_url_2','none')
urls

,URL,Visits,response_code_1,redirect_url_1,response_code_2,redirect_url_2
0,https://www.samsung.com/us/mobile/phones/galax...,11843,none,none,none,none
1,https://www.samsung.com/us/support/televisions...,5667,none,none,none,none
2,https://www.samsung.com/us/mobile/tablets/gala...,5292,none,none,none,none
3,https://www.samsung.com/us/shop/premium-kitche...,5043,none,none,none,none
4,https://www.samsung.com/us/support/contact/+18...,4568,none,none,none,none
...,...,...,...,...,...,...
3078,https://www.samsung.com/us/washers-and-dryers/...,5,none,none,none,none
3079,https://www.samsung.com/us/washers-and-dryers/...,5,none,none,none,none
3080,https://www.samsung.com/us/watches/galaxy-watc...,5,none,none,none,none
3081,https://www.samsung.com/us/watches/galaxy-watch7,5,none,none,none,none


In [180]:
def redirect_checker(url_list):
    rowcount = url_list.URL.shape[0]
    # rowcount = 5
    for row in tqdm(range(rowcount)):
        url = url_list.URL[row]
        try:
            r = requests.get(url)
            # Response Code 1
            if len(r.history) == 0:
                url_list.response_code_1[row] =r.status_code
            else:
                url_list.response_code_1[row] = r.history[0].status_code
                url_list.redirect_url_1[row] = r.url
                if url != r.url:
                        url2 = r.url
                        r2 = requests.get(url2)
                        # Response Code 2
                        url_list.response_code_2[row] = r2.status_code
                        url3 = r2.url
                        if url3 != url2:
                            url_list.redirect_url_2[row] = r2.url
        except:
                print("Error")
return url_list


In [181]:
result = redirect_checker(urls)
result

  0%|          | 0/3083 [00:00<?, ?it/s]

Error


,URL,Visits,response_code_1,redirect_url_1,response_code_2,redirect_url_2
0,https://www.samsung.com/us/mobile/phones/galax...,11843,301,https://www.samsung.com/us/mobile/phones/galax...,404,none
1,https://www.samsung.com/us/support/televisions...,5667,301,https://www.samsung.com/us/support/televisions...,404,none
2,https://www.samsung.com/us/mobile/tablets/gala...,5292,301,https://www.samsung.com/us/tablets/galaxy-tab-...,200,none
3,https://www.samsung.com/us/shop/premium-kitche...,5043,301,https://www.samsung.com/us/shop/premium-kitche...,404,none
4,https://www.samsung.com/us/support/contact/+18...,4568,301,https://www.samsung.com/us/support/contact/+18...,404,none
...,...,...,...,...,...,...
3078,https://www.samsung.com/us/washers-and-dryers/...,5,301,https://www.samsung.com/us/washers-and-dryers/...,404,none
3079,https://www.samsung.com/us/washers-and-dryers/...,5,301,https://www.samsung.com/us/washers-and-dryers/...,404,none
3080,https://www.samsung.com/us/watches/galaxy-watc...,5,301,https://www.samsung.com/us/watches/galaxy-watc...,404,none
3081,https://www.samsung.com/us/watches/galaxy-watch7,5,301,https://www.samsung.com/us/watches/galaxy-watch7/,404,none


In [182]:
# get current date and time as string
current_datetime = datetime.now().strftime("%Y-%m-%d %H-%M")
str_current_datetime = str(current_datetime)

# create a file object along with extension
file_name = "404_check_results_"+ str_current_datetime + ".xlsx"

result.to_excel(file_name)

In [183]:
#URLs to error check
errorcheck_urls = pd.DataFrame()
errorcheck_urls = result[result['response_code_1']=='none']
errorcheck_urls = errorcheck_urls.reset_index(drop=True)
errorcheck_urls


,URL,Visits,response_code_1,redirect_url_1,response_code_2,redirect_url_2
0,www.samsung.com/us,2153,none,none,none,none


: 

In [158]:
#spot checking
errorcheck_result = redirect_checker(errorcheck_urls)
errorcheck_result

# if url != r.url:
#     try:
#         url2 = r.url
#         r2 = requests.get(url2)
#         # Response Code 2
#         url_list.response_code_2[row] = r2.status_code
#         url3 = r2.url
#         if url3 != url2:
#             url_list.redirect_url_2[row] = r2.url
#     except:
#         print("No Second Redirect")

  0%|          | 0/1 [00:00<?, ?it/s]

Error


,URL,Visits,response_code_1,redirect_url_1,response_code_2,redirect_url_2
0,www.samsung.com/us,2153,none,none,none,none
